In [342]:
import pandas as pd
import json
import re
import numpy as np
import ast
from pathlib import Path  

In [343]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_structure_donations/"

In [344]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object']
columns = ['col_path_1','col_path_2','col_path_3','col_path_4', 'col_path_5']


def process_row_path(row, columns, data_types):
    row['data_type'] = ''
    for column in columns:
        if row[column] in data_types:
            row['data_type'] = row[column]
            row[column] = np.nan
        else:
            row[column]
    return row


In [345]:
def file_paths(df):
    # Create different colums for each part of the document path
    df['path_1'] = df['variable'].str.split('/', n=1).str[0]
    df['path_2'] = df['variable'].str.split('/', n=3).str[1]
    df['path_3'] = df['variable'].str.split('/', n=3).str[2]
    df['path_4'] = df['variable'].str.split('/', n=3).str[3]

    # Create a column with the JSON name
    df['json_name'] = df['variable'].str.rsplit('/', n=1).str[-1]


    # As the JSON name is stored in the json_name column, fill other parts of the path with Na if the name of the JSON is present
    mark = ".json"

    df['path_2'] = df['path_2'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)
    df['path_3'] = df['path_3'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)
    df['path_4'] = df['path_4'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)

    # Unlist the value column (where the JSON info is stored)
    for index, row in df.iterrows():
        if isinstance(row['value'], list):
            df.at[index, 'value'] = row['value'][0]
    
    print('file_paths')
    display(df)
    
    return df

In [346]:
def row_column_paths(df):
    # Initialize the new columns
    df['row_path'] = '' 
    df['col_path_1'] = ''  
    df['col_path_2'] = ''  
    df['col_path_3'] = ''
    df['col_path_4'] = ''
    df['col_path_5'] = '' 

   
    # Extract the level 1 keys 
    for index, row in df.iterrows():
        df.at[index, 'row_path'] = list(row['value'].keys())

    print('value', df['value'])
    print('row_path',df['row_path'])

    #df['row_path'] = df['row_path'].astype('str').apply(eval).str[0]
    print('row_path',df['row_path'])


    
    # Take the level 1 keys stored in a list and store them in individual rows
    df = df.explode('row_path')
  

    def column_paths(df, row_var, get_var, unlist_var, explode_var):
        
        print('row_var', df[row_var])
        print('get_var', df[get_var])
        
        # Extract nested values using keys
        df[unlist_var] = df.apply(lambda row: [row[row_var].get(row[get_var], None)] if isinstance (row[row_var], dict) else None, axis=1)

        print('unlist var', df[unlist_var])

        # Unlist and explode
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)
        
        
        

        df[explode_var] = df[unlist_var]
        df = df.explode(explode_var)

    
        
        return df

    row_var = ['value', 'col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values']
    get_var = ['row_path', 'col_path_1', 'col_path_2', 'col_path_3', 'col_path_4']
    unlist_var = ['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values']
    explode_var = ['col_path_1', 'col_path_2', 'col_path_3', 'col_path_4', 'col_path_5']

    for r, g, u, e in zip(row_var, get_var, unlist_var, explode_var):
        df = column_paths(df, r, g, u, e)
        row_var = unlist_var

    return df


In [347]:
def clean_and_store(df, file_name):
 
    # Reorder the columns in the df
    df = df.loc[:, ['variable', 'value', 'path_1', 'path_2',
        'path_3', 'path_4', 'json_name', 'row_path', 'col_path_1','col_path_2','col_path_3', 'col_path_4', 'col_path_5','data_type', 'col_path_1_values','col_path_2_values','col_path_3_values', 'col_path_4_values', 'col_path_5_values']]
    
    df.reset_index(drop=True, inplace=True)
    df = df.fillna('Missing')
    
    


    df.to_csv(f"{main_path}Youtube/Output/Output_" + file_name + '.csv', index=False)
    
    return df

In [348]:
def structure_donations(data):

    data = Path(data)  
    #file_name = data.name  # Extracts "data.json"
    file_name = Path(data).stem 

    # Load JSON file
    with open(data, 'r') as f:
        data = json.load(f)
    

    # Flatten JSON (handling nested structures)
    df = pd.json_normalize(data, max_level=0)

    # Delete user specific informations
    df.columns = df.columns.str.replace(r'^[^/]+/', '', regex=True)

    # Extract column names
    cols = df.columns.tolist()


    # From wide to long df
    df = pd.melt(df, value_vars= cols)

    df = file_paths(df)
    
    df = row_column_paths(df)
    
    df = df.apply(lambda row: process_row_path(row, columns, data_types), axis=1)
    
    df = clean_and_store(df, file_name)
    
    
    return df


In [349]:
input_directory = Path(f'{main_path}Youtube/Input')  
print(input_directory)

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_structure_donations/Youtube/Input


In [350]:

for file in input_directory.iterdir():  
    if file.is_file():  
        print(file)
        structure_donations(file)
        

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_structure_donations/Youtube/Input/json_structure_youtube.json
file_paths


,variable,value,path_1,path_2,path_3,path_4,json_name
0,YouTube and YouTube Music/history/watch-histor...,"{'header': 'string', 'title': 'string', 'title...",YouTube and YouTube Music,history,NaN,NaN,watch-history.json


value 0    {'header': 'string', 'title': 'string', 'title...
Name: value, dtype: object
row_path 0    [header, title, titleUrl, subtitles, time, pro...
Name: row_path, dtype: object
row_path 0    [header, title, titleUrl, subtitles, time, pro...
Name: row_path, dtype: object
row_var 0    {'header': 'string', 'title': 'string', 'title...
0    {'header': 'string', 'title': 'string', 'title...
0    {'header': 'string', 'title': 'string', 'title...
0    {'header': 'string', 'title': 'string', 'title...
0    {'header': 'string', 'title': 'string', 'title...
0    {'header': 'string', 'title': 'string', 'title...
0    {'header': 'string', 'title': 'string', 'title...
Name: value, dtype: object
get_var 0              header
0               title
0            titleUrl
0           subtitles
0                time
0            products
0    activityControls
Name: row_path, dtype: object
unlist var 0                                   [string]
0                                   [string]
0          

In [ ]:
# Path to the folder containing CSV files
output_path = f"{main_path}Youtube/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]

common_columns = ['variable', 'value', 'path_1', 'path_2',
        'path_3', 'path_4', 'json_name', 'row_path', 'col_path_1','col_path_2','col_path_3', 'col_path_4', 'col_path_5','data_type', 'col_path_1_values','col_path_2_values','col_path_3_values', 'col_path_4_values', 'col_path_5_values']


merged_df = dfs[0]  # Start with the first DataFrame
for df in dfs[1:]:  # Merge with the rest  #
    merged_df = merged_df.merge(df, on=common_columns, how="outer")




# Filter where col1 contains 'messages', then drop duplicates based on col2
df_final= merged_df[merged_df["path_1"] == "messages"].drop_duplicates(subset="path_2")


df_final = df_final.replace('Missing', np.nan)


print(df_final.isna().sum())

# Save the final merged DataFrame
df_final.to_csv(f"{main_path}Youtube/Final/Merged_structures_YT.csv", index=False)

variable             0
value                0
path_1               0
path_2               0
path_3               8
path_4               8
json_name            0
row_path             0
col_path_1           6
col_path_2           8
col_path_3           8
col_path_4           8
col_path_5           8
data_type            0
col_path_1_values    0
col_path_2_values    6
col_path_3_values    8
col_path_4_values    8
col_path_5_values    8
dtype: int64


/tmp/ipykernel_1545218/1871176792.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final = df_final.replace('Missing', np.nan)
